<a href="https://colab.research.google.com/github/litlsun/compling/blob/main/gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/litlsun/compling/blob/main/vk_wall_posts.csv

--2024-11-27 09:14:16--  https://github.com/litlsun/compling/blob/main/vk_wall_posts.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘vk_wall_posts.csv’

vk_wall_posts.csv       [ <=>                ] 276.82K  1.44MB/s    in 0.2s    

2024-11-27 09:14:16 (1.44 MB/s) - ‘vk_wall_posts.csv’ saved [283465]



In [2]:
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

In [3]:
with open("vk_wall_posts.csv", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
sentences = [simple_preprocess(sentence) for sentence in text.splitlines() if sentence]

In [5]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

In [15]:
model.wv.most_similar("пожалуйста", topn=5)

[('анон', 0.9997220039367676),
 ('привет', 0.9996791481971741),
 ('всем', 0.9996573328971863),
 ('style', 0.9996071457862854),
 ('false', 0.9994835257530212)]

In [9]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
sentences = [
    [word for word in simple_preprocess(sentence) if word not in stop_words]
    for sentence in text.splitlines() if sentence
]

In [11]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

In [16]:
model.wv.most_similar("пожалуйста", topn=5)

[('анон', 0.9997220039367676),
 ('привет', 0.9996791481971741),
 ('всем', 0.9996573328971863),
 ('style', 0.9996071457862854),
 ('false', 0.9994835257530212)]

In [17]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

In [18]:
dictionary = Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

In [19]:
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [20]:
topics = lda_model.print_topics(num_words=5)
for idx, topic in topics:
    print(f"Topic {idx}: {topic}")

Topic 0: 0.053*"data" + 0.053*"div" + 0.039*"class" + 0.031*"flex" + 0.027*"lg"
Topic 1: 0.069*"li" + 0.051*"svg" + 0.027*"github" + 0.027*"class" + 0.025*"color"
Topic 2: 0.067*"true" + 0.066*"path" + 0.055*"octicon" + 0.049*"zm" + 0.040*"view"
Topic 3: 0.294*"quot" + 0.040*"link" + 0.020*"data" + 0.017*"label" + 0.016*"class"
Topic 4: 0.029*"type" + 0.027*"https" + 0.027*"com" + 0.020*"url" + 0.019*"as"


In [21]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.8 MB/s eta 0:00:00


In [22]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

lda_display = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)

pyLDAvis.display(lda_display)